In [5]:
from ultralytics import YOLO
import cv2
import math
import time

model = YOLO("best.pt")

class_names = {0: 'Basketball', 1: 'Basketball Hoop'}

cap = cv2.VideoCapture("basketball.mp4")

prev_center = None
prev_time = None
basket_count = 0
basket_done = False

while True:
    ret, frame = cap.read()
    if not ret: 
        break
    
    results = model(frame)[0]
    
    basketball_centers = []
    hoop_boxes = []

    for box in results.boxes:
        cls = int(box.cls[0])
        name = class_names.get(cls, f'class_{cls}')
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        
        if name == 'Basketball':
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)
            basketball_centers.append((cx, cy))
            cv2.circle(frame, (cx, cy), 8, (0,0,255), -1)
            cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
        
        elif name == 'Basketball Hoop':
            hoop_boxes.append((x1, y1, x2, y2))
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
            cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

    if basketball_centers:
        center = basketball_centers[0]
        current_time = time.time()

        if prev_center is not None and prev_time is not None:
            dx = center[0] - prev_center[0]
            dy = center[1] - prev_center[1]
            dist = math.sqrt(dx*dx + dy*dy)
            dt = current_time - prev_time
            
            if dt > 0:
                speed = dist / dt
                cv2.putText(frame, f"Speed: {speed:.2f} px/s", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        else:
            speed = 0
        
        prev_center = center
        prev_time = current_time

        for (hx1, hy1, hx2, hy2) in hoop_boxes:
            if hx1 < center[0] < hx2 and hy1 < center[1] < hy2:
                if not basket_done:
                    basket_count += 1
                    basket_done = True
                    print(f"Basket made! Total: {basket_count}")
                cv2.putText(frame, "Basket!", (center[0], center[1] - 30), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,255), 3)
                break
        else:
            basket_done = False

    cv2.putText(frame, f"Total Baskets: {basket_count}", (10, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2)
    
    cv2.imshow("Basket Detection with Speed", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 1 Basketball Hoop, 345.7ms
Speed: 15.6ms preprocess, 345.7ms inference, 23.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Basketball Hoop, 243.6ms
Speed: 20.1ms preprocess, 243.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Basketball, 229.1ms
Speed: 17.3ms preprocess, 229.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Basketball, 285.4ms
Speed: 0.0ms preprocess, 285.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Basketball, 244.8ms
Speed: 12.5ms preprocess, 244.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Basketball, 278.3ms
Speed: 12.7ms preprocess, 278.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Basketball, 254.6ms
Speed: 0.0ms preprocess, 254.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Basketball, 281.4ms
Speed: 0.0ms preprocess, 2